In [1]:
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
def scrape_table_data(soup):
    """Scrape table from a page."""
    # Find the table with a specific role
    table = soup.find("table", attrs={"role": "grid"})

    # Initialize an empty list to store the column names
    column_names = []

    # Find the table header row
    header_row = table.find("tr")

    # Extract the column names from the table header
    for th in header_row.find_all("th"):
        # Process or print the column name
        column_names.append(th.text.strip())

    # Initialize an empty DataFrame
    data = pd.DataFrame(columns=column_names)

    # Iterate over the remaining rows in the table
    for row in table.find_all("tr")[1:]:  # Exclude the header row
        # Extract the cells in each row
        cells = row.find_all("td")

        # Create an empty dictionary to store the cell values
        row_data = {}

        for col_index, cell in enumerate(cells):
            # Process or print the cell content along with its column name
            column_name = column_names[col_index]
            cell_content = cell.text.strip()

            # Add the cell content to the row data dictionary
            row_data[column_name] = cell_content

        # Convert the row data list to a DataFrame with a single row
        row_df = pd.DataFrame([row_data], columns=column_names)

        # Concatenate the row DataFrame with the main DataFrame
        data = pd.concat([data, row_df], ignore_index=True)

        # Select only rows added within 1 week from time of execution

    return data


In [3]:
# Set the options for the ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode, without opening a browser window

# Create an instance of the ChromeDriver
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the initial page
driver.get("https://keyd.gsis.gr/dsae2/iif/faces/pages/static/publicationList.xhtml#")

# Initialize an empty DataFrame
data = pd.DataFrame()

i = 0
# Scrape data from each page
while i<3:
    print(i)
    # Extract the page source and pass it to BeautifulSoup for parsing
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    # Pass the function for scraping the table
    scraped_data = scrape_table_data(soup)

    if i == 0:
        data = scraped_data

    else:
        # Concatenate the new data with the existing data
        data = pd.concat([scraped_data, data], ignore_index=True)

    # Check if there is a next page
    next_page_element = driver.find_element(By.CSS_SELECTOR, "[aria-label='Next Page']")
    next_page_class = next_page_element.get_attribute("class")
    if "disabled" in next_page_class.split():
        break

    # Navigate to the next page
    next_page_element.click()

    time.sleep(2)

    i += 1

# Close the browser and clean up the resources
driver.quit()

# Display the resulting DataFrame
#print(data)

0
1
2


In [11]:
## Get today's date
#today = datetime.now().date()
#
## Calculate the date 7 days ago
#seven_days_ago = today - timedelta(days=7)
#
#data["Ημερομηνία Καταχώρησης"] = pd.to_datetime(data["Ημερομηνία Καταχώρησης"], errors="coerce").dt.date
#prova = data[data["Ημερομηνία Καταχώρησης"] >= seven_days_ago]